<a href="https://colab.research.google.com/github/MonikaPandey0706/Hate_Speech_Detection_Project/blob/main/Hate_Speech_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [23]:
import numpy as np
import pandas as pd
import nltk

import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from nltk.corpus import stopwords
from nltk.util import pr
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk

nltk.download('stopwords')

stopwords = set(stopwords.words('english'))
stemmer = PorterStemmer()



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
df = pd.read_csv('Dataset---Hate-Speech-Detection-using-Deep-Learning.csv')
print(df.head(10))

   class                                              tweet
0      2  !!! RT @mayasolovely: As a woman you shouldn't...
1      1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2      1  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3      1  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4      1  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5      1  !!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just...
6      1  !!!!!!"@__BrighterDays: I can not just sit up ...
7      1  !!!!&#8220;@selfiequeenbri: cause I'm tired of...
8      1  " &amp; you might not get ya bitch back &amp; ...
9      1  " @rhythmixx_ :hobbies include: fighting Maria...


# New Section

In [25]:
df["labels"] = df["class"].map({
    0: "Hate Speech",
    1: "Offensive Speech",
    2: "Not Offensive"
})
print(df.head(10))

   class                                              tweet            labels
0      2  !!! RT @mayasolovely: As a woman you shouldn't...     Not Offensive
1      1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  Offensive Speech
2      1  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  Offensive Speech
3      1  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  Offensive Speech
4      1  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  Offensive Speech
5      1  !!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just...  Offensive Speech
6      1  !!!!!!"@__BrighterDays: I can not just sit up ...  Offensive Speech
7      1  !!!!&#8220;@selfiequeenbri: cause I'm tired of...  Offensive Speech
8      1  " &amp; you might not get ya bitch back &amp; ...  Offensive Speech
9      1  " @rhythmixx_ :hobbies include: fighting Maria...  Offensive Speech


In [26]:
df=df[["tweet","labels"]]
print(df.head(10))

                                               tweet            labels
0  !!! RT @mayasolovely: As a woman you shouldn't...     Not Offensive
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  Offensive Speech
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  Offensive Speech
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  Offensive Speech
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  Offensive Speech
5  !!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just...  Offensive Speech
6  !!!!!!"@__BrighterDays: I can not just sit up ...  Offensive Speech
7  !!!!&#8220;@selfiequeenbri: cause I'm tired of...  Offensive Speech
8  " &amp; you might not get ya bitch back &amp; ...  Offensive Speech
9  " @rhythmixx_ :hobbies include: fighting Maria...  Offensive Speech


In [27]:
df['tweet'] = df['tweet'].str.lower()

# Removing punctuations present in the text
punctuations_list = string.punctuation
def remove_punctuations(text):
    temp = str.maketrans('', '', punctuations_list)
    return text.translate(temp)

df['tweet']= df['tweet'].apply(lambda x: remove_punctuations(x))
df.head()

,tweet,labels
0,rt mayasolovely as a woman you shouldnt compl...,Not Offensive
1,rt mleew17 boy dats coldtyga dwn bad for cuff...,Offensive Speech
2,rt urkindofbrand dawg rt 80sbaby4life you eve...,Offensive Speech
3,rt cganderson vivabased she look like a tranny,Offensive Speech
4,rt shenikaroberts the shit you hear about me ...,Offensive Speech


In [28]:

def clean(text):
  text=str(text).lower()
  text=re.sub('\[.*?\]','',text)
  text=re.sub('https?://\S+|www\.\S+','',text)
  text=re.sub('<.*?>+','',text)
  text=re.sub('[%s]' % re.escape(string.punctuation),'',text)
  text=re.sub('\n','',text)
  text=re.sub('\w*\d\w*','',text)
  #text=" ".join(text)
  text=[stemmer.stem(word) for word in text.split(' ')]
  text=" ".join(text)
  return text

df["tweet"]=df["tweet"].apply(clean)
df['tweet'].replace('', np.nan, inplace=True)
df.dropna(subset=['tweet'], inplace=True)



<ipython-input-28-81e3f2588beb>:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['tweet'].replace('', np.nan, inplace=True)


In [29]:
x = np.array(df["tweet"])
y = np.array(df["labels"])
if len(x) == 0:
    print("Error: All documents became empty after cleaning.")
else:
    cv = CountVectorizer()
    try:
        x = cv.fit_transform(x)

        # Add a check for empty vocabulary after fit_transform
        if len(cv.vocabulary_) == 0:
            print("Error: CountVectorizer resulted in an empty vocabulary. Check cleaning steps.")
        else:
            X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
            clf = DecisionTreeClassifier()
            clf.fit(X_train, y_train)
            print("Model trained successfully.")

    except ValueError as e:
        print(f"Error during CountVectorizer fit_transform: {e}")


Model trained successfully.


In [30]:
test_data="I will Kill You!!"
df=cv.transform([test_data]).toarray()
print(clf.predict(df))

['Hate Speech']
